<a href="https://colab.research.google.com/github/ttjh1234/Study_MLDL/blob/main/chapter07_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 심층 신경망

인공 신경망에 층을 여러 개 추가하여 패션 MNIST dataset을 분류하면서 

케라스로 심층 신경망을 만드는 방법을 공부한다.

## 2개의 층

다시 케라스 API를 사용해서 패션 MNIST 데이터셋을 불러온다.

그다음 인공신경망을 적용시키기 위해 전처리를 진행하겠다.

일단 이미지 픽셀값을 0~1사이로 변환 시키고, 28*28 배열 2차원 배열을 

784크기의 1차원 배열로 만든후, train_set과 test_set으로 나눈다.

In [ ]:
from tensorflow import keras
(x_train,y_train),(x_test,y_test)=keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
from sklearn.model_selection import train_test_split
x_train_scaled=x_train/255.0
x_train_scaled=x_train_scaled.reshape(-1,28*28)
x_train_scaled,x_valid,y_train,y_valid=train_test_split(x_train_scaled,y_train,test_size=0.2,random_state=42)

이제 인공 신경망 모델에 층을 2개 추가해 보겠다.

1절에 만든 신경망 모델과 다른 점은 입력층과 출력층 사이에 밀집층이 추가된 것이다.

입력층과 출력층 사이에 있는 모든 층을 **은닉층**이라고 한다.

은닉층에는 활성화 함수가 표시되어있다. 신경망 층의 선형 방정식의 계산 값에 적용하는 함수이다.

이전 절에서 출력층에 적용했던 softmax 함수도 활성화 함수이다.

하지만 출력층에 적용하는 활성화 함수는 종류가 제한되어있다.

**이진 분류**일 경우 **sigmoid** 함수를 적용하고, **다중 분류**일 경우 **softmax** 함수를 적용한다.

이에 비해 은닉층의 활성화 함수는 비교적 자유롭다. 

대표적으로 **sigmoid** 함수와 **relu** 함수 등을 사용한다.



---
은닉층에 활성화 함수를 적용하는 이유는 선형 계산을 적당하게 

**비선형적**으로 만들어 주는 역할을 하기 때문이다.

은닉층의 활성화 함수로 sigmoid 함수를 사용하여 신경망을 만들어보겠다.

In [ ]:
dense1=keras.layers.Dense(100,activation='sigmoid',input_shape=(784,))
dense2=keras.layers.Dense(10,activation='softmax')

dense1이 은닉층이고 100개의 뉴런을 가진 밀집층이다.

활성화 함수로 sigmoid 함수를 사용하였고, 

input_shape는 784개의 픽셀값을 받기 위해 (784,)로 지정하였다.

뉴런의 수는 특별한 기준이 없지만, 한 가지 제약 조건이 있다.

적어도 출력층의 뉴런보다는 많게 만들어야 한다.

클래스 10개에 대한 확률을 예측하는데, 은닉층의 뉴런이 10개보다 적으면 

적은 정보가 전달될 것이기 때문이다.

dense2는 출력층으로 10개의 클래스를 분류하므로 10개의 뉴런을 두었고, 

활성화 함수로 softmax 함수를 사용하였다.

## 심층 신경망 만들기

앞에서 만든 은닉층과 출력층을 Sequential 클래스에 대입하여 **심층 신경망**을 만들어 보겠다.

Sequential 클래스의 객체를 만들 때, 여러 개의 층을 추가하려면 리스트로 만들어 전달을 하는데,

이 때 출력층을 마지막에 두어야한다. 즉 순서를 생각해서 리스트를 만들어야 한다. 

In [ ]:
model=keras.Sequential([dense1,dense2])

케라스는 모델의 summary() 메서드를 호출하면 층에 대한 유용한 정보를 얻을 수 있다.

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               78500     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


맨 첫 줄에 모델의 이름이 나온다. 

그다음 층이 순서대로 나열되는데, 층마다 층 이름, 클래스, 출력 크기, 모델 파라미터 개수가 출력된다.

출력 크기를 보면 (None,100)인데, 첫 번째 차원은 샘플의 개수를 나타낸다.

샘플 개수가 아직 정의되어 있지 않기 때문에, None이다. 

케라스 모델의 fit() 메서드에 훈련 데이터를 주입하면 이 데이터를 한 번에 모두 사용하지 않고,

경사 하강법 단계를 수행한다.

케라스의 미니배치 크기는 32인데, fit() 메서드에서 **batch_size** 매개변수로 바꿀 수 있다.

마지막에는 총 모델 파라미터 개수와 훈련되는 파라미터 개수와 아직 훈련되지 않은 파라미터 개수를 출력한다.

간혹 경사 하강법으로 훈련되지 않는 파라미터를 가진 층이 있다.

## 층을 추가하는 다른 방법

앞에서는 Dense 클래스의 객체 dense1,dense2를 만들어 Sequential 클래스에 전달했다.

하지만 따로 저장해서 사용할 일이 없기 때문에, Sequential 클래스 내부에서 정의해준다.

In [ ]:
model=keras.Sequential([
                        keras.layers.Dense(100,activation='sigmoid',input_shape=(784,),name='hidden'),
                        keras.layers.Dense(10,activation='softmax',name='output')
],name='패션 MNIST 모델')

In [ ]:
model.summary()

Model: "패션 MNIST 모델"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden (Dense)               (None, 100)               78500     
_________________________________________________________________
output (Dense)               (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


이 방법이 편리하지만, 아주 많은 층을 추가하려면 문장이 길어지고, 

또 조건에 따라 층을 추가할 수 없다.

Sequential 클래스에서 층을 추가할 때 널리 사용하는 방법은 모델의 **add()** 메서드이다.

In [ ]:
model=keras.Sequential()
model.add(keras.layers.Dense(100,activation='sigmoid',input_shape=(784,)))
model.add(keras.layers.Dense(10,activation='softmax'))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 100)               78500     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='sparse_categorical_crossentropy',metrics='accuracy')
model.fit(x_train_scaled,y_train,epochs=5)

Epoch 1/5
1500/1500 [==============================] - 6s 2ms/step - loss: 0.7619 - accuracy: 0.7572
Epoch 2/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.4102 - accuracy: 0.8536
Epoch 3/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3748 - accuracy: 0.8644
Epoch 4/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3536 - accuracy: 0.8707
Epoch 5/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3323 - accuracy: 0.8800


## 렐루 함수

초창기 인공 신경망의 은닉층에 많이 사용된 활성화 함수는 sigmoid 함수였다.

이 함수는 오른쪽과 왼쪽끝으로 갈수록 그래프가 누워있어 올바른 출력을 만드는데 한계가 있다.

특히 층이 많은 심층 신경망일수록 그 효과가 누적되어 학습을 더 어렵게 만든다.

이를 개선하기 위해 **Relu** 함수를 사용한다.

렐루 함수는 특히 이미지 처리에서 좋은 성능을 낸다고 알려져있다.

패션 MNIST 데이터는 이미지 픽셀이 28*28 크기이기 때문에, 인공 신경망에 주입하기 위해

넘파이 배열의 reshape 메서드를 사용하여 1차원 배열로 만들어주었다.

케라스에서는 Flatten 층을 제공하여 reshape 변환 작업을 대신한다.


In [ ]:
model=keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add(keras.layers.Dense(100,activation='relu'))
model.add(keras.layers.Dense(10,activation='softmax'))

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)               78500     
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


Flatten 클래스에 포함된 모델 파라미터는 0개이다.

케라스의 Flatten 층을 신경망 모델에 추가하면, 입력값의 차원을 짐작할 수 있다는 것이 장점이다.

다시 train_set을 준비해서 모델을 훈련하겠다.

In [ ]:
(x_train,y_train),(x_test,y_test)=keras.datasets.fashion_mnist.load_data()
x_train_scaled=x_train/255.0
x_train_scaled,x_valid,y_train,y_valid=train_test_split(x_train_scaled,y_train,test_size=0.2,random_state=42)

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',metrics='accuracy')
model.fit(x_train_scaled,y_train,epochs=5)

Epoch 1/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.6799 - accuracy: 0.7595
Epoch 2/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3980 - accuracy: 0.8585
Epoch 3/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3572 - accuracy: 0.8705
Epoch 4/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3260 - accuracy: 0.8826
Epoch 5/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3134 - accuracy: 0.8876


In [ ]:
model.evaluate(x_valid,y_valid)

375/375 [==============================] - 1s 2ms/step - loss: 0.3789 - accuracy: 0.8738


[0.37891069054603577, 0.8738333582878113]

전 장에서 한 개의 은닉층을 수행한 것 보다 더 정확도가 향상한 것을 볼 수 있다.

## 옵티마이저

하이퍼 파라미터는 모델이 학습하지 않아 사람이 지정해주어야 하는 파라미터이다.

은닉층의 개수, 뉴런 개수, 활성화 함수, 층의 종류, 배치 사이즈 매개변수, 에포크 매개변수 등 

여러가지 하이퍼 파라미터가 있다.

또한 complie() 메소드의 경사 하강법 알고리즘도 정해주어야 한다.

지금까지 위에서 사용한 complie()메소드의 경사 하강법 알고리즘은 기본인 **RMSprop**을 사용했다.

케라스는 다양한 종류의 경사 하강법 알고리즘을 제공하는데, 이들을 **옵티마이저**라고 한다.

여기서는 여러가지 옵티마이저를 사용하겠다.

가장 기본적인 옵티마이저는 확률적 경사 하강법인 SGD 이다.

SGD 옵티마이저를 사용하는 방법은 2가지가 있다.

In [ ]:
model.compile(optimizer='sgd',loss='sparse_categorical_crossentropy',metrics='accuracy')

In [ ]:
sgd=keras.optimizers.SGD()
model.compile(optimizer=sgd,loss='sparse_categorical_crossentropy',metrics='accuracy')

SGD 클래스의 학습률 기본값은 0.01 인데 이를 바꾸고자 한다면, 

원하는 학습률을 **learning_rate** 매개변수에 지정하여 사용한다.

In [ ]:
sgd=keras.optimizers.SGD(learning_rate=0.1)

SGD 외에도 다양한 옵티마이저들이 있다. 

기본 경사 하강법 옵티마이저는 모두 SGD 클래스에서 제공한다.

SGD 클래스의 **momentum** 매개변수의 기본값은 0이다.

이를 0 보다 큰 값으로 지정하면 마치 이전의 그레이디언트를 가속도처럼 사용하는 

**모멘텀 최적화**를 사용한다.

보통 momentum 매개변수는 0.9 이상을 지정한다.

SGD 클래스의 **nesterov** 매개변수를 기본값 False 에서 True 로 바꾸면

**네스테로프 모멘텀 최적화**를 사용한다.

네스테로프 모멘텀은 모멘텀 최적화를 2번 반복하여 구현한다.

대부분의 경우 기본 확률적 경사 하강법보다 더 나은 성능을 제공한다.

모델이 최적점에 가까이 갈수록 학습률을 낮출 수 있다. 이렇게 하면 안정적으로 최적점에 수렴할 가능성이 높다.

이런 학습률은 **적응적 학습률** 이라고 한다.

적응적 학습률을 사용하는 대표적인 옵티마이저는 **Adagrad**와 **RMSprop**이다.


In [ ]:
adagrad=keras.optimizers.Adagrad()
model.compile(optimizer=adagrad,loss='sparse_categorical_crossentropy',metrics='accuracy')

rmsprop=keras.optimizers.RMSprop()
model.compile(optimizer=rmsprop,loss='sparse_categorical_crossentropy',metrics='accuracy')

모멘텀 최적화와 RMSprop의 장점을 접목한 것이 **Adam** 이다.

적응적 학습률을 사용하는 이 3개의 클래스는 learning_rate 매개변수의 

기본값으로 모두 0.001을 사용한다.



In [ ]:
model=keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add(keras.layers.Dense(100,activation='relu'))
model.add(keras.layers.Dense(10,activation='softmax'))

In [ ]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics='accuracy')
model.fit(x_train_scaled,y_train,epochs=5)

Epoch 1/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.6832 - accuracy: 0.7610
Epoch 2/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.4133 - accuracy: 0.8522
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3583 - accuracy: 0.8686
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3302 - accuracy: 0.8787
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3093 - accuracy: 0.8885


In [ ]:
model.evaluate(x_valid,y_valid)

375/375 [==============================] - 1s 2ms/step - loss: 0.3336 - accuracy: 0.8810


[0.33355772495269775, 0.8809999823570251]